<a href="https://colab.research.google.com/github/ChandanCharchit/NLP/blob/main/Abstractive%20News%20Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

directory = "/content/drive/MyDrive/Scrap_assignment"
filename = "aus_news.txt"

os.makedirs(directory, exist_ok=True)
full_path = os.path.join(directory, filename)


In [ ]:
import requests
from bs4 import BeautifulSoup

try:
  url = 'https://www.abc.net.au/news/business'

  response = requests.get(url)

  soup = BeautifulSoup(response.content, 'html.parser')

  articles = soup.find_all('a', class_='GenericCard_link__Crwfx Link_link__nE06W ScreenReaderOnly_srLinkHint__83_S_ Link_showVisited__gmCxW Link_showFocus__0kDeK Link_underlineNone__W1VuH')
  content = []
  for article in articles:
      article_url = 'https://www.abc.net.au' + article['href']
      article_response = requests.get(article_url)
      article_soup = BeautifulSoup(article_response.content, 'html.parser')
      article_title = article_soup.find('h1', class_='Typography_base__k7c9F Heading_heading__XLh_j Typography_sizeMobile24__j8FWA Typography_sizeDesktop36__nwYOG Typography_lineHeightMobile32__sHfSN Typography_lineHeightDesktop48__w_kN7 Typography_marginBottomMobileSmall__8rIrY Typography_marginBottomDesktopSmall__IsBSx Typography_black__5rKXY Typography_colourInherit__xnbjy Typography_normalise__UWWOc').get_text(strip=True)
      article_paragraphs = article_soup.find_all('p', class_='paragraph_paragraph__3Hrfa')
      article_content = '\n'.join([p.get_text() for p in article_paragraphs])
      scraped_data = f'Title: {article_title}\n'+ '-' * 50 + '\n' + f'Content: {article_content}\n' + '\n' +'-' * 50 + '\n'

      content.append(scraped_data)
except ConnectionError as e:
  pass
except AttributeError as ea:
  pass

In [ ]:
with open(full_path, "w") as file:
  for item in content:
    file.write(item)

In [ ]:
pip install transformers datasets

In [ ]:
import torch
from transformers import DistilBertForTokenClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        padding="max_length",
        max_length=128, 
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
num_labels = len(set(tag for dataset in dataset.values() for example in dataset for tag in example['ner_tags']))
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.139000,0.050025
2,0.030400,0.046293
3,0.011800,0.048140


TrainOutput(global_step=2634, training_loss=0.0487122087652428, metrics={'train_runtime': 494.3845, 'train_samples_per_second': 85.203, 'train_steps_per_second': 5.328, 'total_flos': 1376049275709696.0, 'train_loss': 0.0487122087652428, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])

In [ ]:
def prepare_input(text, max_length=128):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > max_length - 2:  # [CLS] and [SEP] tokens
        tokens = tokens[: max_length - 2]

    input_ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"])
    input_tensor = torch.tensor([input_ids])
    return input_tensor


In [ ]:
def label_to_entity_name(predictions, input_ids, label_list):
    pred_labels = torch.argmax(predictions, dim=2)[0].tolist()
    entities = []
    for idx, label in enumerate(pred_labels):
        if label != -100:
            entity = label_list[label]
            token = tokenizer.convert_ids_to_tokens(input_ids[idx])
            entities.append((token, entity))
    return entities

In [ ]:
def predict(text, model, label_list):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    input_tensor = prepare_input(text)
    input_tensor = input_tensor.to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    predictions = output.logits
    entities = label_to_entity_name(predictions.cpu(), input_tensor[0].tolist(), label_list)
    return entities

In [ ]:
import pandas as pd


In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
data = []
token_label = []
df = {}
for item in content:
  entities = predict(item, model, label_list)
  for prediction in entities:
    # if prediction[1] != 'O':
    data.append(prediction[0])
    token_label.append(prediction[1])

df['tokens'] = data
df['labels'] = token_label

final_df = pd.DataFrame(df)

In [ ]:
final_df.head()

,tokens,labels
0,[CLS],O
1,title,O
2,:,O
3,as,B-ORG
4,##x,I-ORG


In [ ]:
final_df

,tokens,labels
0,[CLS],O
1,title,O
2,:,O
3,as,B-ORG
4,##x,I-ORG
...,...,...
2555,cent,O
2556,.,O
2557,both,O
2558,the,O


In [ ]:
final_df.to_csv('news_tagged_with_O.csv')

Topic modelling: Extractive Summarization

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
metric = load_metric("rouge")

# def prepare_data(examples):
#     inputs = tokenizer(examples["article"], padding="max_length", truncation=True, max_length=512)
#     labels = tokenizer(examples["highlights"], padding="max_length", truncation=True, max_length=128)
#     inputs["labels"] = labels["input_ids"]
#     return inputs


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
def prepare_data(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    labels = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


tokenized_dataset = dataset.map(prepare_data, batched=True, batch_size=4)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
  pip install rouge_score

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    save_strategy="steps",
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    eval_steps=500,
    save_steps=500,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_scores = metric.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rouge1"])
    return rouge_scores

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [ ]:
trainer.train()


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: ignored

In [ ]:
def generate_summary(text, model, tokenizer, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


In [ ]:
summary = generate_summary(text, model, tokenizer)
print(summary)

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset


In [ ]:
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def tokenize_data(example):
    inputs = tokenizer(example['article'], truncation=True, padding='max_length', max_length=512)
    outputs = tokenizer(example['highlights'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = outputs['input_ids']
    return inputs

dataset = dataset.map(tokenize_data, batched=True)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)


In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.808400
1000,1.415100
1500,1.375200
2000,1.388600
2500,1.325700
3000,1.327200
3500,1.286200
4000,1.348000
4500,1.330900
5000,1.293100


KeyboardInterrupt: ignored

In [ ]:
def generate_summary(text, model, tokenizer, max_length=128):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(model.device)

    with torch.no_grad():to
        summary_ids = model.generate(inputs, max_length=max_length, num_beams=5, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:

text = '''A company that makes podcasts discussing explicit sex acts and interviewing porn stars won a $382,712 contract from the Office of the Prime Minister and Cabinet under the Morrison government.
The contract, detailed in published reports, is among other taxpayer-funded deals revealed, including:
Tomorrow night's federal budget has to balance a lot of competing demands, with millions of people feeling the impact of a cost of living crisis.
"We'll have government spending of roughly in the realm of $650 billion. So that's a lot of money that is being spent," says Committee for Economic Development of Australia (CEDA) senior economist Cassandra Winzar.
Budgets — particularly multi-billion-dollar programs — reveal the true priorities of governments.
For Ms Winzar, the bigger issue is a lack of checks or what's called "evaluation" of programs.
Essentially, is what we're paying for doing what we hoped?
"Look, it's incredibly concerning," she says.
"A lot of that money is coming from taxpayers and they should be wanting to know how that money is being spent, and what impact it's having."
The federal government might dole out nearly $650,000,000,000 in spending annually, but it all adds up.
Between December 2020 and November 2022, taxpayers gave $382,712 to a Queensland company for "communications" services — dealing with social media content relating to COVID-19.'''
summary = generate_summary(text, model, tokenizer)
print(summary)

The contract is among other taxpayer-funded deals revealed.
The federal government has to balance a lot of competing demands, including millions of people feeling the impact of a cost of living crisis.
"Alot of that money is coming from taxpayers"
